# Pasturland analysis

In [ ]:
import os 
del os.environ['GDAL_DATA']

In [ ]:
import itertools

import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, calculate_default_transform as cdt, Resampling
from rasterstats import zonal_stats
from geocube.api.core import make_geocube
from shapely.geometry import shape, box
from matplotlib.colors import ListedColormap
from matplotlib import cm
import ee
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd

from sepal_ui import mapping as sm

from scripts import parameters as pm
from scripts.utils import update_progress

In [ ]:
ee.Initialize()

Map = sm.SepalMap(['CartoDB.Positron'])
Map

## LMICs

### get the countries borders as ee.FeatureCollection

Get the LMICs countries from the GAUL list and display them on the interactive map

In [ ]:
countries = pm.country_list
countries.head()

In [ ]:
gaul_numbers = countries['GAUL'].tolist()
ee_countries = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.inList('ADM0_CODE', gaul_numbers))
Map.addLayer(ee_countries, {}, 'LMIC')

### transform to shapely

The ee collection need to be transformed into a list of shapely shape to be used as a croping border
need to use a list of polygon because a single polygon exceed the maximum number of vertice

In [ ]:
shapes = []
for index, row in countries.iterrows():
    update_progress(index / (len(countries)-1), 'Countries loaded')
    country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', row.GAUL))
    country_json = geemap.ee_to_geojson(country)
    shapes.append(shape(country_json['features'][0]['geometry']))

## Pasturland map 

Using the map provided by Yelena we will crop the raster on the LMICs map

### cut the map of pastureland on the countries

In [ ]:
with rasterio.open(pm.pastureLand_raster) as src:
    out_image, out_transform = mask(src, shapes, all_touched=True)
    out_meta = src.meta.copy()
    out_meta.update(
        driver = 'GTiff',
        height = out_image.shape[1],
        width = out_image.shape[2],
        transform = out_transform
    )
    with rasterio.open(pm.pasture_masked, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
Map.add_raster(pm.pasture_masked, layer_name='pasture')

## convert the TNC ecoregions into WWF biomes and rasterize it 

TNC terrestrial ecoregion that can be found [here](https://geospatial.tnc.org/datasets/7b7fb9d945544d41b3e7a91494c42930_0?showData=true)

### get the biomes in the WWF shapefile

In [ ]:
tnc_glob = gpd.read_file(pm.tnc_ecozone_shp) 
tnc_glob.head()

In [ ]:
%%time

convert_to_rast = tnc_glob[['BIOME', 'geometry']] 
convert_to_rast = convert_to_rast.dissolve(by='BIOME')
convert_to_rast = convert_to_rast.reset_index()
convert_to_rast.head()

In [ ]:
%%time

convert_to_rast['BIOME'] = convert_to_rast['BIOME'].astype(int)

with rasterio.open(pm.pastureLand_raster) as src:
    gt = src.transform

    resX = gt[0]
    resY = -gt[4]
    
    cube = make_geocube(
        convert_to_rast,
        measurements=["BIOME"],
        resolution=(resX, resY),
    )

In [ ]:
cube['BIOME'].rio.to_raster(pm.wwf_biomes)

In [ ]:
%%time 

with rasterio.open(pm.wwf_biomes) as src:
    out_image, out_transform = mask(src, shapes, all_touched=True)
    out_meta = src.meta.copy()

    out_meta.update(
        driver = 'GTiff',
        height = out_image.shape[1],
        width = out_image.shape[2],
        transform = out_transform
    )

    with rasterio.open(pm.biomes_masked, "w", **out_meta) as dst:
        dst.write(out_image)

In [ ]:
#gcreate a colormp for the display in interactive map
vals = convert_to_rast['BIOME'].unique()
tab20 = cm.get_cmap('tab20', len(vals))
tab20 = tab20(np.linspace(0, 1, len(vals)))

color_map = ListedColormap(tab20, N=int(vals.max())+1)

In [ ]:
Map.add_raster(pm.biomes_masked, layer_name='ecozones', colormap=color_map)

## deal with the 2000 LLC zones
from the 2000 map of LLC
Determine the fraction of the cell’s area in the land cover classes in ESA’s CCI-LC maps (100, 110, 120, 130, and 150)
each zone will be treated separately to avoid bugs

### functions 

In [ ]:
#transform the raster into a gpd cell grid 
def get_grid(src_rst, max_polygon=0):

    with rasterio.open(src_rst) as src:
        data = src.read(1)

        t = src.transform

        move_x = t[0]
        # t[4] is negative, as raster start upper left 0,0 and goes down
        # later for steps calculation (ymin=...) we use plus instead of minus
        move_y = t[4]

        height = src.height
        width = src.width 

        polygons = []
        indices = list(itertools.product(range(height), range(width)))
        index = 0
        data_flat = data.flatten()
        for y,x in indices:
            if data_flat[index] != np.iinfo(np.int16).min:
                x_min, y_max = t * (x,y)
                x_max = x_min + move_x
                y_min = y_max + move_y
                polygons.append(box(x_min, y_min, x_max, y_max))
            
            index += 1
            
            if (max_polygon != 0) & (len(polygons) > max_polygon):
                break

    return gpd.GeoDataFrame(crs='EPSG:4236', geometry=polygons)

In [ ]:
def gdf_zonal_stats(init_gdf, rst):
    
    gdf = init_gdf.copy()
    gdf = gdf.join(
        pd.DataFrame(
            zonal_stats(
                vectors=gdf['geometry'], 
                raster=rst, 
                all_touched=True, 
                categorical=True
            )
        ),
        how='left'
    )
    
    return gdf

In [ ]:
def fraction_raster(init_gdf, value, template_rst, out_rst):
    
    gdf = init_gdf.copy()
    gdf['{}%'.format(value)] = gdf[value]/gdf.sum(axis=1)*100
    
    with rasterio.open(template_rst) as src:
        gt = src.transform
        resX = gt[0]
        resY = -gt[4]
        
    cube = make_geocube(
        gdf,
        measurements=['{}%'.format(value)],
        resolution=(resX, resY),
    )
    
    cube['{}%'.format(value)].rio.to_raster(out_rst)

In [ ]:
def align_raster(src_rst, template_rst, out_rst):
    
    # get template crs and transform 
    with rasterio.open(template_rst) as tplt, rasterio.open(src_rst) as src:      
        
        kwargs = src.meta.copy()
        kwargs.update(
            driver = 'GTiff',
            height = tplt.height,
            width = tplt.width,
            transform = tplt.transform
        )
        
        #destination
        with rasterio.open(out_rst, 'w', **kwargs) as dst:
            for i in range(1, dst.count+1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=tplt.transform,
                    dst_crs=tplt.crs,
                    resampling=Resampling.bilinear
                ) 

### get the grid 

This grid will be used as a base for each ecozone value 
sheck the gdf_zonal header to verify that your ecozone is in the polygons regions (not always the case when using max_polygon)

In [ ]:
# value of max polygone
# comment to create the full map
# the 200 first values are placed in the north of kazakstan
max_polygon = 200

In [ ]:
%%time 
gdf_grid = get_grid(pm.pasture_masked, max_polygon=max_polygon)

In [ ]:
print('{} cells in gdf_grid'.format(len(gdf_grid)))

In [ ]:
%%time
gdf_zonal = gdf_zonal_stats(gdf_grid, pm.llc_2000_raster)
gdf_zonal.head()

### create the ecozones 

In [ ]:
ecozones = [100, 110, 120, 130, 150]

for index, zone in enumerate(ecozones):
    
    update_progress(index / (len(ecozones)-1), 'ecozones loaded')
    
    if zone in gdf_zonal.columns:
        fraction_raster(gdf_zonal, zone, pm.pasture_masked, pm.llc_2000_map.format(zone))
        align_raster(pm.llc_2000_map.format(zone), pm.pasture_masked.format(zone), pm.llc_2000_map_masked.format(zone))
        Map.add_raster(pm.llc_2000_map_masked.format(zone), layer_name=pm.ecozones[zone])

In [ ]:
len(gdf_grid)